In [ ]:
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
from matplotlib import pyplot as plt
import numpy as np
import math
import os, sys, glob, re
import time
import scanpy as sc
import anndata
import scipy
import time
import math
import seaborn as sns

## Firstly, load h5ad

In [ ]:
dataDir = './'
adata = sc.read_h5ad(dataDir+'/seurat_int_logTotal_10000.h5ad')

In [ ]:
adata

## Secondly, clustering with leiden

In [ ]:
%%time
sc.pp.pca(adata)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50, save="01.pca_variance_ratio.png")

In [ ]:
%%time
sc.pp.neighbors(adata, n_pcs=20)
sc.tl.umap(adata)

In [ ]:
%%time
sc.tl.leiden(adata, resolution=0.35, key_added="clusters_reso_0.35")

In [ ]:
%%time
sc.tl.leiden(adata, resolution=1, key_added="clusters_reso_1")

In [ ]:
sc.pl.umap(adata, color=['nCount_RNA', 'nFeature_RNA', "clusters_reso_0.35"], wspace=0.4, save="02-umap-reso0.35.png")

In [ ]:
sc.pl.umap(adata, color=['nCount_RNA', 'nFeature_RNA', "clusters_reso_1"], wspace=0.4, save="02-umap-reso1.png")

In [ ]:
# display the spatial for each sample
samples = adata.obs['id'].unique()
selected_samples = ['CRCP59_T_2', 'CRCP95_T']
for sample in selected_samples:
    subset_adata = adata[adata.obs['id'] == sample]
    sc.pl.spatial(subset_adata, color=["clusters_reso_0.35"], spot_size=1, show=None, size=1.2, title=f'Sample: {sample}')

## Thirdly, display heatmap for marker genes

In [ ]:
# nc_genes from the original paper
nc_gene=['CEACAM1','EPCAM','SCD','KRT19','SELENOP','IL1B','KRT18','SPINK1','GPX2','OLFM4','MKI67','TOP2A','PATJ','IGF2R','CXCL14','MS4A1','CXCL8','TRDC','TRAC','CCL5','TIGIT','SPP1','CCR7','CD86','LTB','CXCR5','LYZ','IL7R','CD84','GZMK','ITGAX','LAPTM5','CD69','KLRK1','JCHAIN','IGHA1','IGHG4','PDGFRA','GPM6B','RORA','MYH11','PI16','COL1A1','COL1A2','COL6A3','TPSB2','PCDHA6','EGR1','IGFBP7','PECAM1','NOTCH3','FABP4','GATA2','TIMP3','P4HA3']
gene = [item for item in nc_gene if item in adata.var_names]

In [ ]:
len(nc_gene)

In [ ]:
len(gene) # How many markers used in the paper are found in the selected 10000 features? 44 out of 55

In [ ]:
# the heatmap for the clusters in the origianl paper
order=['epi_CEA', 'epi_MKI67', 'epi_normal', 'boundary', 'fiber and cavity', 
       'immune-infiltrated stroma', 'immune_plasma_cell', 'smooth_muscle_IGFBP5', 
       'smooth_muscle_RGS5', 'smooth_muscle_DES', 'stroma_ECM', 'stroma_COL1A1', 
       'stroma_EGR1', 'stroma_vessel', 'stroma_TIMP3']
sc.pl.matrixplot(adata, gene, groupby='level2', categories_order=order, standard_scale='var', cmap='inferno', save='_fig1b.pdf')

In [ ]:
# the heatmap for the clusters generated by our script
cluster_order = ['0','1','2','14','12','11','6','13','9','5','4','3','8','7','10']
sc.pl.matrixplot(adata, gene, groupby='clusters_reso_1', categories_order=cluster_order, standard_scale='var', cmap='inferno', save='our_fig1b.pdf')